In [ ]:
import hmftpy as hmf
from hmftpy.plaquettes.triangular import plaq12, plaq12z, plaq9p, plaq9d, plaq3
from hmftpy.operators import mf_ops, inner_hamiltonian, periodic_hamiltonian
from hmftpy import do_hmft
from hmftpy.hmf import get_mfs
from hmftpy.plot_spins import draw_tri_lattice, draw_cluster, plot_spins
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

plaq = plaq12
L = plaq['L']
basis = spin_basis_1d(L, pauli=0)

ops = hmf.operators.mf_ops(plaq, basis)
maxit=50

In [ ]:
steps = 60

energies = {'c1': np.zeros(steps), 'c2': np.zeros(steps), 'm1': np.zeros(steps)}
J2s = np.linspace(1, 0.25, steps)
mf0s = {'c1': None, 'c2': None, 'm1': None, 'inner': None}
cvgs = {'c1': [False for i in range(steps)], 
        'c2': [False for i in range(steps)],
        'm1': [False for i in range(steps)],
        'inner': [False for i in range(steps)]
       }

def couplings(J2, alpha=0, h=0):
    interactions = {'local': {'x': h},
                    'nearest': {'xx': 1, 'yy': 1, 'zz': alpha},
                    'n_nearest': {'xx': J2, 'yy': J2, 'zz': alpha*J2},
                    'n_n_nearest': {}}
    return interactions

In [ ]:
v0 = None
mf0s['m1'] = None
for i, J2 in enumerate(tqdm(J2s, ascii=True)):
    interactions = couplings(J2, alpha=1, h=0)
    energies['m1'][i], v, mf, cvgs['m1'][i] = do_hmft(plaq, interactions, basis, 
                                            max_iter=maxit, mf0=mf0s['m1'],
                                            ops=ops,
                                            mf_cvg=True, every_other=True, hmft_tol=10**-10,
                                            v0=v0)
    # if J2 > 0.5:
    mf0s['m1'] = mf
    if np.sum(np.abs(mf['x'])) + np.sum(np.abs(mf['y'])) < 10**-8:
        print('Zero mean field at {}'.format(J2))
        # break
    v0 = v

In [ ]:
energies['inner'] = np.zeros(steps)         
for i, J2 in enumerate(tqdm(J2s, ascii=True)):
    interactions = couplings(J2, alpha=1, h=0)
    h = inner_hamiltonian(plaq, interactions, basis, every_other=True)
    e, v = h.eigsh(k=1, which='SA')
    energies['inner'][i] = e[0]

# Looking at how shit works

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
plt.scatter(J2s[cvgs['m1']], energies['m1'][cvgs['m1']], marker='.')
plt.plot(J2s, energies['inner'])